In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os

In [ ]:
import tqdm

# BERT Encoding

## pip installs

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.3 MB/s 
     |████████████████████████████████| 182 kB 62.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.9 MB/s 


## BERT model loading

In [ ]:
# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt
%matplotlib inline

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
def bert_embedding(sentence, model):
  marked_text = "[CLS] " + sentence + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)

      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2] 
      token_embeddings = torch.stack(hidden_states, dim=0)
      token_embeddings = torch.squeeze(token_embeddings, dim=1)
      token_embeddings = token_embeddings.permute(1,0,2) # n_tokens x n_layers (13) x 768

      #sum the last four hidden layers for each word to get word embeddings
      # n_tokens x 768
      word_embeddings = torch.sum(token_embeddings[:,-4:-1,:], dim=1)
      # average all word embeddings to get sentence embeddings
      sentence_embedding=torch.mean(word_embeddings, dim=0) # 768 dim vector

      return sentence_embedding

In [ ]:
# DATA_PATH='/content/drive/My Drive/MS US/CSCI 544 NLP/CSCI 544 PROJ GROUP 44/data/'
os.chdir('..')
DATA_PATH=os.getcwd()+'/data/'

In [ ]:
# loading scraped movie plots
with open(DATA_PATH+'sentenceWikiTopGrossing.txt', "r") as wiki:
  dataset = json.loads(wiki.read())

In [ ]:
#indices of considered movies
movie_list=['Indiana Jones and the Kingdom of the Crystal Skull', \
            "Harry Potter and the Philosopher's Stone (film)", \
            'Skyfall', \
            'Thor: Ragnarok', \
            'Star Wars: Episode III – Revenge of the Sith', \
            'Forrest Gump', \
            'Jumanji: Welcome to the Jungle']
movies_req=[]
for movie in movie_list:
  movies_req.append(dataset['Movies'].index(movie))
movies_req

[78, 31, 63, 28, 94, 79, 13]

In [ ]:
data = dict()
movies = list()
plots = list()
for i in movies_req:
  movies.append(dataset['Movies'][i])
  plot = list()
  for scene in dataset['Plots'][i]:
    plot.append(scene)
  plots.append(plot)
data['Movies'] = movies
data['Plots'] = plots

In [ ]:
data['Movies']

['Indiana Jones and the Kingdom of the Crystal Skull',
 "Harry Potter and the Philosopher's Stone (film)",
 'Skyfall',
 'Thor: Ragnarok',
 'Star Wars: Episode III – Revenge of the Sith',
 'Forrest Gump',
 'Jumanji: Welcome to the Jungle']

In [ ]:
data['Movies'][0], data['Plots'][0] 

('Indiana Jones and the Kingdom of the Crystal Skull',
 ['In 1957, Soviet KGB agents, working under Irina Spalko, kidnap Indiana Jones and his partner George "Mac" MicHale.',
  'The Soviets infiltrate a secret government warehouse, located in Nevada, labeled "Hangar 51", and force Jones to locate a mummified alien corpse from the Roswell UFO incident, which he was forced to work on 10 years earlier.',
  'Shortly after retrieving the corpse, Mac reveals he has become a double agent on the KGB’s payroll.',
  "Jones unsuccessfully attempts to steal the body, and fights with Spalko's henchman Dovchenko, before escaping to a nearby model town right before an atomic bomb test.",
  'He takes shelter in a lead-lined refrigerator, and FBI agents eventually rescue, decontaminate, and interrogate him, suspecting him of working for the Soviets.',
  'Though eventually freed, Jones is put on an indefinite leave of absence from Marshall College, and dean Charles Stanforth resigns to spare Jones from 

In [ ]:
length = len(data['Movies'])
length

7

In [ ]:
scene_embeddings = dict()
for i in range(length):
  print(data['Movies'][i])
  embedding = list()
  for scene in tqdm.tqdm(data['Plots'][i]):
    embedding.append(bert_embedding(scene, model))
  scene_embeddings[data['Movies'][i]] = embedding

Indiana Jones and the Kingdom of the Crystal Skull


100%|██████████| 27/27 [00:05<00:00,  4.82it/s]


Harry Potter and the Philosopher's Stone (film)


100%|██████████| 37/37 [00:04<00:00,  7.65it/s]


Skyfall


...

100%|██████████| 30/30 [00:04<00:00,  6.70it/s]


Forrest Gump


100%|██████████| 36/36 [00:04<00:00,  7.85it/s]


Jumanji: Welcome to the Jungle


100%|██████████| 25/25 [00:04<00:00,  6.13it/s]


In [ ]:
scene_embeddings['Skyfall'][0].shape

torch.Size([768])

In [ ]:
import pickle

with open(DATA_PATH+"scene_embeddings_bert_7.pkl", "wb+") as plots:
  pickle.dump(scene_embeddings, plots)

In [ ]:
import pickle
with open(DATA_PATH+"scene_embeddings_bert_7.pkl", "rb") as plots:
  scene_embeddings=pickle.load(plots)

## Experimenting with some custom queries

### eg 1: Indiana Jones and the Kingdom of the Crystal Skull

In [ ]:
user_query = "It's the height of the Cold War, and famous archaeologist Indiana Jones, returning from his latest adventure, finds out his job at Marshall College is in jeopardy"

In [ ]:
user_embedding = bert_embedding(user_query, model)
user_embedding.shape

torch.Size([768])

In [ ]:
user_embedding.reshape(1,-1).shape

torch.Size([1, 768])

In [ ]:
comparison = dict()
for key in scene_embeddings: #for each movie
  maximum = -np.inf
  for scene in scene_embeddings[key]: #each scene in movie
    output = torch.cosine_similarity(user_embedding.reshape(1,-1), scene.reshape(1,-1))
    output = float(output[0])
    if output>maximum:
      maximum = output 
  comparison[key] = maximum #highest similarity scene found for each movie

In [ ]:
comparison

{'Indiana Jones and the Kingdom of the Crystal Skull': 0.8032756447792053,
 "Harry Potter and the Philosopher's Stone (film)": 0.761189877986908,
 'Skyfall': 0.7568788528442383,
 'Thor: Ragnarok': 0.7619942426681519,
 'Star Wars: Episode III – Revenge of the Sith': 0.7199779748916626,
 'Forrest Gump': 0.7748528122901917,
 'Jumanji: Welcome to the Jungle': 0.781177282333374}

In [ ]:
rank = list(sorted(comparison.items(), key=lambda item: item[1], reverse = True))
rank

[('Indiana Jones and the Kingdom of the Crystal Skull', 0.8032756447792053),
 ('Jumanji: Welcome to the Jungle', 0.781177282333374),
 ('Forrest Gump', 0.7748528122901917),
 ('Thor: Ragnarok', 0.7619942426681519),
 ("Harry Potter and the Philosopher's Stone (film)", 0.761189877986908),
 ('Skyfall', 0.7568788528442383),
 ('Star Wars: Episode III – Revenge of the Sith', 0.7199779748916626)]

### eg 2: Harry Potter and the Philosopher's Stone (film)

In [ ]:
user_query = "An orphan boy goes to a magical school." 
user_embedding = bert_embedding(user_query, model)

In [ ]:
comparison = dict()
for key in scene_embeddings:
  maximum = -np.inf
  for scene in scene_embeddings[key]:
    output = torch.cosine_similarity(user_embedding.reshape(1,-1), scene.reshape(1,-1))
    output = float(output[0])
    if output>maximum:
      maximum = output
  comparison[key] = maximum

In [ ]:
rank = list(sorted(comparison.items(), key=lambda item: item[1], reverse = True))
rank

[('Forrest Gump', 0.7484495043754578),
 ("Harry Potter and the Philosopher's Stone (film)", 0.7174956798553467),
 ('Indiana Jones and the Kingdom of the Crystal Skull', 0.7045598030090332),
 ('Thor: Ragnarok', 0.6981692314147949),
 ('Jumanji: Welcome to the Jungle', 0.696295440196991),
 ('Skyfall', 0.6797429323196411),
 ('Star Wars: Episode III – Revenge of the Sith', 0.6612194776535034)]

### eg 3: Forrest Gump

In [ ]:
user_query = "An innocent and kind-hearted Alabama boy, has been dealing with other people's unkindness nearly all his life. Having grown up with beautiful Jenny, his only friend, Forrest yearns to learn all about the ways of the world and embarks on a mission to find his true purpose in life."
user_embedding = bert_embedding(user_query, model)

In [ ]:
comparison = dict()
for key in scene_embeddings:
  maximum = -np.inf
  for scene in scene_embeddings[key]:
    output = torch.cosine_similarity(user_embedding.reshape(1,-1), scene.reshape(1,-1))
    output = float(output[0])
    if output>maximum:
      maximum = output
  comparison[key] = maximum

In [ ]:
rank = list(sorted(comparison.items(), key=lambda item: item[1], reverse = True))
rank

[('Forrest Gump', 0.8187644481658936),
 ('Jumanji: Welcome to the Jungle', 0.816012978553772),
 ('Skyfall', 0.7851630449295044),
 ('Indiana Jones and the Kingdom of the Crystal Skull', 0.7716839909553528),
 ("Harry Potter and the Philosopher's Stone (film)", 0.7652992606163025),
 ('Thor: Ragnarok', 0.7636489272117615),
 ('Star Wars: Episode III – Revenge of the Sith', 0.7583550214767456)]

# BERT Score

## pip installs and model loading

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 451 kB 52.9 MB/s 
     |████████████████████████████████| 115 kB 77.2 MB/s 
     |████████████████████████████████| 212 kB 72.0 MB/s 
     |████████████████████████████████| 127 kB 33.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.7 MB/s 


In [ ]:
from evaluate import load
bertscore = load("bertscore")

# https://huggingface.co/spaces/evaluate-metric/bertscore
# test inputs
predictions = ["hello world", "general kenobi"]
references = ["goodnight moon"]*len(predictions) 
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.789967954158783, 0.5812705755233765], 'recall': [0.789967954158783, 0.6331430077552795], 'f1': [0.789967954158783, 0.6060988903045654], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.24.0)'}


In [ ]:
results['f1'][0]

0.789967954158783

## Experimenting with example queries

### eg 1: Indiana Jones and the Kingdom of the Crystal Skull

In [ ]:
user_query = "It's the height of the Cold War, and famous archaeologist Indiana Jones, returning from his latest adventure, finds out his job at Marshall College is in jeopardy"

In [ ]:
bertscore_comparison1 = dict()
for i in range(length):
  print(data['Movies'][i])
  maximum = -np.inf
  for scene in data['Plots'][i]:
    output = bertscore.compute(predictions=[scene], references=[user_query], model_type="distilbert-base-uncased")
    output = float(output['f1'][0])
    if output>maximum:
      maximum = output
  bertscore_comparison1[data['Movies'][i]] = maximum

Indiana Jones and the Kingdom of the Crystal Skull
Harry Potter and the Philosopher's Stone (film)
Skyfall
Thor: Ragnarok
Star Wars: Episode III – Revenge of the Sith
Forrest Gump
Jumanji: Welcome to the Jungle


In [ ]:
bertscore_comparison1

{'Indiana Jones and the Kingdom of the Crystal Skull': 0.7456532120704651,
 "Harry Potter and the Philosopher's Stone (film)": 0.7078917622566223,
 'Skyfall': 0.7275628447532654,
 'Thor: Ragnarok': 0.7257481813430786,
 'Star Wars: Episode III – Revenge of the Sith': 0.7106615304946899,
 'Forrest Gump': 0.7304571270942688,
 'Jumanji: Welcome to the Jungle': 0.7175719738006592}

In [ ]:
bertscore_rank = list(sorted(bertscore_comparison1.items(), key=lambda item: item[1], reverse = True))
bertscore_rank

[('Indiana Jones and the Kingdom of the Crystal Skull', 0.7456532120704651),
 ('Forrest Gump', 0.7304571270942688),
 ('Skyfall', 0.7275628447532654),
 ('Thor: Ragnarok', 0.7257481813430786),
 ('Jumanji: Welcome to the Jungle', 0.7175719738006592),
 ('Star Wars: Episode III – Revenge of the Sith', 0.7106615304946899),
 ("Harry Potter and the Philosopher's Stone (film)", 0.7078917622566223)]

### eg 2: Harry Potter and the Philosopher's Stone (film)

In [ ]:
user_query = "An orphan boy goes to a magical school."

In [ ]:
bertscore_comparison2 = dict()
for i in range(length):
  print(data['Movies'][i])
  maximum = -np.inf
  for scene in data['Plots'][i]:
    output = bertscore.compute(predictions=[scene], references=[user_query], model_type="distilbert-base-uncased")
    output = float(output['f1'][0])
    if output>maximum:
      maximum = output
  bertscore_comparison2[data['Movies'][i]] = maximum

Indiana Jones and the Kingdom of the Crystal Skull
Harry Potter and the Philosopher's Stone (film)
Skyfall
Thor: Ragnarok
Star Wars: Episode III – Revenge of the Sith
Forrest Gump
Jumanji: Welcome to the Jungle


In [ ]:
bertscore_rank = list(sorted(bertscore_comparison2.items(), key=lambda item: item[1], reverse = True))
bertscore_rank

[("Harry Potter and the Philosopher's Stone (film)", 0.7778573632240295),
 ('Forrest Gump', 0.7320542335510254),
 ('Jumanji: Welcome to the Jungle', 0.7180209755897522),
 ('Indiana Jones and the Kingdom of the Crystal Skull', 0.7089331150054932),
 ('Skyfall', 0.7061852812767029),
 ('Thor: Ragnarok', 0.7038484215736389),
 ('Star Wars: Episode III – Revenge of the Sith', 0.6875035762786865)]

### eg 3: Forrest Gump

In [ ]:
user_query = "An innocent and kind-hearted Alabama boy, has been dealing with other people's unkindness nearly all his life. Having grown up with beautiful Jenny, his only friend, Forrest yearns to learn all about the ways of the world and embarks on a mission to find his true purpose in life."

In [ ]:
bertscore_comparison3 = dict()
for i in range(length):
  print(data['Movies'][i])
  maximum = -np.inf
  for scene in data['Plots'][i]:
    output = bertscore.compute(predictions=[scene], references=[user_query], model_type="distilbert-base-uncased")
    output = float(output['f1'][0])
    if output>maximum:
      maximum = output
  bertscore_comparison3[data['Movies'][i]] = maximum

Indiana Jones and the Kingdom of the Crystal Skull
Harry Potter and the Philosopher's Stone (film)
Skyfall
Thor: Ragnarok
Star Wars: Episode III – Revenge of the Sith
Forrest Gump
Jumanji: Welcome to the Jungle


In [ ]:
bertscore_rank = list(sorted(bertscore_comparison3.items(), key=lambda item: item[1], reverse = True))
bertscore_rank

[('Jumanji: Welcome to the Jungle', 0.7426944971084595),
 ('Forrest Gump', 0.7403373718261719),
 ("Harry Potter and the Philosopher's Stone (film)", 0.7324876189231873),
 ('Thor: Ragnarok', 0.7310616970062256),
 ('Indiana Jones and the Kingdom of the Crystal Skull', 0.7251374125480652),
 ('Star Wars: Episode III – Revenge of the Sith', 0.7216182947158813),
 ('Skyfall', 0.7181929349899292)]

# Testing

## Paraphrased Paragraph test set

In [ ]:
test_scenes = pd.read_csv(DATA_PATH+"/test_data/ParaphrasedScenes_test.csv", usecols=['Movies', 'Plots'])
test_scenes

,Movies,Plots
0,Thor: Ragnarok,"Two years after the battle of Sokovia, Thor is..."
1,Thor: Ragnarok,After discovering that his brother was pretend...
2,Thor: Ragnarok,She chases the two as they attempt to flee thr...
3,Thor: Ragnarok,A slave trader named Scrapper 142 subdues him ...
4,Thor: Ragnarok,"After escaping the palace, he found the Quinje..."
...,...,...
119,Jumanji: Welcome to the Jungle,"Twenty-one years later, four students from the..."
120,Jumanji: Welcome to the Jungle,"Spencer theorizes that if they lose all three,..."
121,Jumanji: Welcome to the Jungle,"Van Pelt's men, Spencer and Fridge's animosity..."
122,Jumanji: Welcome to the Jungle,"While heading for the shrine, the group work t..."


In [ ]:
test_scenes['Movies'][8]

'Thor: Ragnarok'

#### BERT + cosine

In [ ]:
from collections import defaultdict
para_scene_embeddings = defaultdict(list)
for i in range(len(test_scenes['Movies'])):
  embedding = bert_embedding(test_scenes['Plots'][i], model)
  para_scene_embeddings[test_scenes['Movies'][i]].append(embedding)

In [ ]:
para_scene_embeddings['Thor: Ragnarok'][0].shape

torch.Size([768])

In [ ]:
# import pickle

# with open(DATA_PATH+"para_scenes_embeddings.pkl", "wb+") as plots:
#   pickle.dump(para_scene_embeddings, plots)

In [ ]:
scene_embeddings['Thor: Ragnarok'][0].shape

torch.Size([768])

In [ ]:
#top-1 accuracy count
answer = {"Correct": 0, "Wrong": 0}
count=0
for query_key in tqdm.tqdm(para_scene_embeddings): #each movie (i.e. correct label of test query)
  for query_scene in para_scene_embeddings[query_key]: #embeddings of each test query
    count+=1
    maximum = -np.inf
    answer_key = ""
    for key in scene_embeddings:
      for scene in scene_embeddings[key]:
        output = torch.cosine_similarity(query_scene.reshape(1,-1), scene.reshape(1,-1))
        output = float(output[0])
        if output>maximum:
          maximum = output
          answer_key = key
    if answer_key==query_key:
      answer['Correct'] += 1
    else:
      answer['Wrong'] += 1

100%|██████████| 5/5 [00:00<00:00,  5.28it/s]


In [ ]:
answer

{'Correct': 122, 'Wrong': 2}

In [ ]:
answer['Correct']/(answer['Correct']+answer['Wrong'])  # accuracy

0.9838709677419355

#### BERTScore

In [ ]:
from collections import defaultdict
para_scenes = defaultdict(list)
for i in range(len(test_scenes['Movies'])):
  # print(test_scenes['Movies'][i])
  embedding = test_scenes['Plots'][i]
  print(embedding)
  para_scenes[test_scenes['Movies'][i]].append(embedding)

In [ ]:
para_scenes['Thor: Ragnarok'][1]

'After discovering that his brother was pretending to be his father, and with the help of Stephen Strange at the Sanctum Sanctorum in New York City, they locate their father in Norway.'

In [ ]:
import pickle

with open(DATA_PATH+"para_scenes.txt", "w+") as plots:
  plots.write(json.dumps(para_scenes))

In [ ]:
#top-1 accuracy count
answer = {"Correct": 0, "Wrong": 0}
for query_key in tqdm.tqdm(para_scenes): #each movie (i.e. correct label of test query)
  for query_scene in para_scenes[query_key]: #embeddings of each test query
    maximum = -np.inf
    answer_key = ""
    # rank_dict=dict()

    
    for i in range(length):
      # print(data['Movies'][i])
      key=data['Movies'][i]
      # print(key)
      # maximum = -np.inf
      for scene in data['Plots'][i]:

        output = bertscore.compute(predictions=[scene], references=[query_scene], model_type="distilbert-base-uncased") #can eliminate one for loop here and pass all scenes directly
        output = float(output['f1'][0])

        if output>maximum:
          maximum = output
          answer_key = key

      # rank_dict[key] = maximum
    # print(rank_dict)
    # rank_dict_top3=set( movie for (movie, score) in list(sorted(rank_dict.items(), key=lambda item: item[1], reverse = True))[:3])
    # print('final',query_key, rank_dict_top3)

    if query_key==answer_key:
      answer['Correct'] += 1
    else:
      answer['Wrong'] += 1

100%|██████████| 5/5 [05:24<00:00, 64.85s/it]


In [ ]:
answer

{'Correct': 124, 'Wrong': 0}

In [ ]:
answer['Correct']/(answer['Correct']+answer['Wrong'])  # accuracy

1.0

## Paraphrased Sentences test set

In [ ]:
test_sentences = pd.read_csv(DATA_PATH+"/test_data/ParaphrasedSentences_test.csv", usecols=['Movies', 'Plots'])
test_sentences

,Movies,Plots
0,Indiana Jones and the Kingdom of the Crystal S...,"In 1957, Soviet KGB agents kidnapped Indiana J..."
1,Indiana Jones and the Kingdom of the Crystal S...,The Soviets force Jones to locate a mummified ...
2,Indiana Jones and the Kingdom of the Crystal S...,Mac became a double agent after retrieving the...
3,Indiana Jones and the Kingdom of the Crystal S...,"Jones tried to steal the body, but was defeate..."
4,Indiana Jones and the Kingdom of the Crystal S...,"He takes shelter in a lead-lined refrigerator,..."
...,...,...
595,Jumanji: Welcome to the Jungle,The Vreeke home was restored and decorated for...
596,Jumanji: Welcome to the Jungle,"Alex, now an adult, told the group that he was..."
597,Jumanji: Welcome to the Jungle,He has a son named Andy and a daughter named B...
598,Jumanji: Welcome to the Jungle,"Spencer and Fridge patching things up, and Spe..."


#### BERT + cosine

In [ ]:
sent_scene_embeddings = defaultdict(list)
for i in range(len(test_sentences['Movies'])):
  # print(test_sentences['Movies'][i])
  embedding = bert_embedding(test_sentences['Plots'][i], model)
  sent_scene_embeddings[test_sentences['Movies'][i]].append(embedding)

In [ ]:
sent_scene_embeddings['Thor: Ragnarok'][0].shape

torch.Size([768])

In [ ]:
# import pickle

# with open(DATA_PATH+"para_scenes_embeddings.pkl", "wb+") as plots:
#   pickle.dump(para_scene_embeddings, plots)

In [ ]:
scene_embeddings['Thor: Ragnarok'][0].shape

torch.Size([768])

In [ ]:
#top-1 accuracy count
answer = {"Correct": 0, "Wrong": 0}
count=0
for query_key in tqdm.tqdm(sent_scene_embeddings): #each movie (i.e. correct label of test query)
  for query_scene in sent_scene_embeddings[query_key]: #embeddings of each test query
    count+=1
    answer_key = ""
    maximum = -np.inf
    answer_key = ""
    for key in scene_embeddings:
      for scene in scene_embeddings[key]:
        output = torch.cosine_similarity(query_scene.reshape(1,-1), scene.reshape(1,-1))
        output = float(output[0])
        if output>maximum:
          maximum = output
          answer_key = key
    if answer_key==query_key:
      answer['Correct'] += 1
    else:
      answer['Wrong'] += 1
print(count)

100%|██████████| 5/5 [00:04<00:00,  1.19it/s]

600


In [ ]:
answer

{'Correct': 595, 'Wrong': 5}

In [ ]:
answer['Correct']/(answer['Correct']+answer['Wrong'])  # accuracy

0.9916666666666667

#### BERTScore

In [ ]:
from collections import defaultdict
sent_scenes = defaultdict(list)
for i in range(len(test_sentences['Movies'])):
  # print(test_sentences['Movies'][i])
  embedding = test_sentences['Plots'][i]
  print(embedding)
  sent_scenes[test_sentences['Movies'][i]].append(embedding)

In [ ]:
sent_scenes['Thor: Ragnarok'][1]

'He says that the realm will be destroyed when the Eternal Flame burns in the vault.'

In [ ]:
import pickle

with open(DATA_PATH+"sent_scenes.txt", "w+") as plots:
  plots.write(json.dumps(sent_scenes))

In [ ]:
#top-1 accuracy count
answer = {"Correct": 0, "Wrong": 0}
for query_key in tqdm.tqdm(sent_scenes): #each movie (i.e. correct label of test query)
  for query_scene in sent_scenes[query_key]: #embeddings of each test query
    maximum = -np.inf
    answer_key = ""
    
    for i in range(length):
      key=data['Movies'][i]
      for scene in data['Plots'][i]:
        output = bertscore.compute(predictions=[scene], references=[query_scene], model_type="distilbert-base-uncased") #can eliminate one for loop here and pass all scenes directly
        output = float(output['f1'][0])

        if output>maximum:
          maximum = output
          answer_key = key

    if query_key==answer_key:
      answer['Correct'] += 1
    else:
      answer['Wrong'] += 1

100%|██████████| 5/5 [25:18<00:00, 303.72s/it]


In [ ]:
answer

{'Correct': 600, 'Wrong': 0}

In [ ]:
answer['Correct']/(answer['Correct']+answer['Wrong'])  # scenes

1.0